In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

## Scrape Table from the Website

In [2]:
# pandas.read_html(): The Shortcut
dfs = pd.read_html("http://www.aastocks.com/en/cnhk/market/quota-balance/hk-connect")

In [3]:
# Manual find the data that we need
cap_data = dfs[12:][1]

In [4]:
# drop one level (Southbound Trade) from the columns
cap_data = cap_data.droplevel(0, axis=1) 

# drop another level from the columns
cap_data = cap_data.droplevel(1, axis=1) 

# Data Cleaning

## Daily Quota Balance(% of Quota)

In [5]:
# Extract the Parentheses into a separate column
Percent_of_Quota = []
for i in cap_data["Daily Quota Balance(% of Quota)"]:
    if i == "Closed":
        Percent_of_Quota.append(0)
    else:
        Percent_of_Quota.append(i.split("(")[1])
        
# clean up the Percent_of_Quota to make it clean list
Percent_of_Quota = [float(j[0:-2]) if j != 0 else 0 for j in Percent_of_Quota]

# add the list into the dataframe
# insert the column in a specific column

cap_data.insert(2, "Percentage of Quota", Percent_of_Quota)

In [6]:
# extract the daily quota balance from the column
Daily_Quota_Balance = []
for k in cap_data["Daily Quota Balance(% of Quota)"]:
    if k == "Closed":
        Daily_Quota_Balance.append(0)
    else:
        Daily_Quota_Balance.append(float(k.split("(")[0][:-1]))
        
# replace the column with the newly extracted list
cap_data["Daily Quota Balance(% of Quota)"] = Daily_Quota_Balance

# rename the column name
cap_data.rename(columns={"Daily Quota Balance(% of Quota)": "Daily Quota Balance(in Billion)",}, inplace=True)

## Money Flow

In [7]:
# create a new list to store clean data for money flow
money_flow = []
for l in cap_data["Money Flow"]:
    if l == "-":
        money_flow.append(0)
        
    # change the in to positive number and out to negative number
    # divide number in million by 1,000 to make the number in same scale
    
    elif l.split(" ")[0] == "In": 
        
        if "M" in l.split(" ")[1]:
            money_flow.append(float(l.split(" ")[1][:-1])/1000)
        else:
            money_flow.append(float(l.split(" ")[1][:-1]))
            
    else:
        if "M" in l.split(" ")[1]:
            money_flow.append((float(l.split(" ")[1][:-1])/1000)*-1)
        else:
            money_flow.append(float(l.split(" ")[1][:-1])*-1)
            
# replace the column with the newly extracted list
cap_data["Money Flow"] = money_flow

# rename the column name
cap_data.rename(columns={"Money Flow": "Money Flow(in Billion)",}, inplace=True)

## Buy Trade Value & Sell Trade Value

In [8]:
# create a new list to store clean data for money flow
buy_trade = []
for m in cap_data["Buy Trade Value(HKD)"]:
    if m == "-":
        buy_trade.append(0)
    else:
        buy_trade.append(float(m[:-1]))
        
# replace the column with the newly extracted list
cap_data["Buy Trade Value(HKD)"] = buy_trade

# create a new list to store clean data for money flow
sell_trade = []
for n in cap_data["Sell Trade Value(HKD)"]:
    if n == "-":
        sell_trade.append(0)
    else:
        sell_trade.append(float(n[:-1]))
        
# replace the column with the newly extracted list
cap_data["Sell Trade Value(HKD)"] = sell_trade

## Total Trade Value

In [10]:
# Extract the Parentheses into a separate column
mkt_turnover = []
for o in cap_data["Total Trade Value1 (% of Market Turnover)"]:
    if o == "-":
        mkt_turnover.append(0)
    else:
        mkt_turnover.append(o.split("(")[1])
        
# clean up the Percent_of_Quota to make it clean list
mkt_turnover = [float(o[0:-2]) if o != 0 else 0 for o in mkt_turnover]

# add the list into the dataframe
cap_data["Percentage of Market Turnover"] = mkt_turnover

In [12]:
# extract the total trade value from the column
Total_Trade_Value = []
for p in cap_data["Total Trade Value1 (% of Market Turnover)"]:
    if p == "-":
        Total_Trade_Value.append(0)
    else:
        Total_Trade_Value.append(float(p.split("(")[0][:-1]))
        
# replace the column with the newly extracted list
cap_data["Total Trade Value1 (% of Market Turnover)"] = Total_Trade_Value

# rename the column name
cap_data.rename(columns={"Total Trade Value1 (% of Market Turnover)": "Total Trade Value(in Billion)",}, inplace=True)

# Export the table into Excel Folder

In [33]:
# set the excel file name
current_date = "".join(cap_data["Date"][0].split("/"))

# export the table as an excel file
cap_data.to_excel(current_date+".xlsx")